# Retailrocket: E-commerce Purchase Prediction - Data Cleaning
---

## Problem Statement

An e-commerce company wishes to understand visitors behaviour on its site and the factors that drive these site visitors to make a purchase. This is so that they can better target the right audience and improve future online sales. 

For the aim of this project, the Retailrocket e-commerce dataset from <a href="https://www.kaggle.com/retailrocket/ecommerce-dataset">Kaggle</a> is used. The goal is to build and compare different binary classification models that would best predict the purchasing intention of visitors to this e-commerce site. The results and findings will then be presented to the stakeholders in the e-commerce company.

## Executive Summary

The notebook for the Capstone project is broken down into 2 parts. The first part (this notebook) covers the data cleaning for each of the 4 datasets that have been provided on <a href="https://www.kaggle.com/retailrocket/ecommerce-dataset">Kaggle</a> - Item Properties 1, Item Properties 2, Category Tree and Events. 

Most of the product features in the Item Properties datasets are encrypted except for category id and product availability. Hence, only these two features could be used. 

After cleaning, these 4 datasets are then merged and grouped to form a dataset that is based on session-level. This dataset will then be used in the next notebook for further feature engineering, data visualisations and modelling. 

### Contents:
- [1. Import Libraries](#1.-Import-Libraries)
- [2. Data Import](#2.-Data-Import)
- [3. Data Cleaning](#3.-Data-Cleaning)
    - [3.1. Item Properties Datasets](#3.1.-Item-Properties-Datasets)
    - [3.2. Category Tree Dataset](#3.2.-Category-Tree-Dataset)
    - [3.3. Merged Items and Category Datasets](#3.3.-Merged-Items-and-Category-Datasets)
    - [3.4. Events Dataset](#3.4.-Events-Dataset)
    - [3.5. Merged Events, Items and Category Datasets](#3.5.-Merged-Events,-Items-and-Category-Datasets)

## 1. Import Libraries
---

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#import plotly.express as px
#import plotly.graph_objects as go

from datetime import datetime, timedelta

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, confusion_matrix, accuracy_score, precision_score, recall_score, fbeta_score
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity
from scipy.sparse import csr_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from scipy.stats import mode
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer

## 2. Data Import
---

In [3]:
pd.set_option("display.max_columns", None)

In [4]:
# read the data in chunks
def read_data_chunk(data_path, chunksize):
    data_chunk = pd.read_csv(data_path, chunksize=chunksize)
    
    chunk_list = []
    for chunk in data_chunk:
        
        chunk_list.append(chunk)
    
    dataframe = pd.concat(chunk_list)
    
    return dataframe

In [5]:
# apply the function to read the data
events = read_data_chunk(("../data/retailrocket/events.csv"), 100000)
item1 = read_data_chunk(("../data/retailrocket/item_properties_part1.csv"), 100000)
item2 = read_data_chunk(("../data/retailrocket/item_properties_part2.csv"), 100000)
category_tree = read_data_chunk(("../data/retailrocket/category_tree.csv"), 100000)

In [13]:
# display events
events.head(2)

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN


In [14]:
# display category_tree
category_tree.head(2)

,categoryid,parentid
0,1016,213.0
1,809,169.0


In [15]:
# display item1
item1.head(2)

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200


In [16]:
# display item2
item2.head(2)

,timestamp,itemid,property,value
0,1433041200000,183478,561,769062
1,1439694000000,132256,976,n26.400 1135780


## 3. Data Cleaning
---

## 3.1. Item Properties Datasets

There are 2 Item Properties datasets and these datasets provide the details of the items (for example, the category identifier of an item). Each row in the dataset specifies the detail of a specific property of an item at a particular point in time. 

Since the properties of an item may change over time, these item properties are time dependent. For example, an item id can have different category id over time. Hence, the item id and its corresponding category id may appear multiple times in the dataset. Furthermore, an item may also have the same properties over time and only appear once in the datasets. Hence, for further merging of these datasets, the time factor is also included and taken into consideration.

Additionally, all values in these 2 datasets are hashed except "categoryid" and "available" properties. Hence, for the aim of this project, only the data for these 2 values are used and the rest are removed. The cleaning of these 2 datasets will be broken down into 3 parts:

- Item - Category
- Item - Availability
- Item - Merge

The first two covers the cleaning of the two properties with respect to the datasets. Meanwhile, the last part covers the merging of the datasets.

### Item - Category

In [9]:
# clean item-category datasets
def clean_catid(dataframe):
    dataframe = dataframe[dataframe["property"] == "categoryid"].reset_index(drop=True)
    dataframe["value"] = dataframe["value"].astype("int")
    dataframe = dataframe.rename(columns={"value": "categoryid"})
    
    return dataframe

In [10]:
# clean item-availability datasets
def clean_avail(dataframe):
    dataframe = dataframe[dataframe["property"] == "available"].reset_index(drop=True)
    dataframe["value"] = dataframe["value"].astype("int")
    dataframe = dataframe.rename(columns={"value": "is_available"})
    
    return dataframe

In [11]:
# create a function to drop duplicates
def drop_duplicates(dataframe):
    dataframe = dataframe.drop_duplicates(subset=dataframe.columns.tolist(), keep="first")
    
    return dataframe

In [12]:
# create a function to convert timestamp to various time-related features
def convert_to_datetime(dataframe):
    datetime_list = []
    for i, unix in enumerate(dataframe["timestamp"]):
        timestamp, ms = divmod(unix, 1000)
        date_time = datetime.fromtimestamp(timestamp) - timedelta(hours=5)
        formatted_datetime = date_time.strftime("%Y-%m-%d %H:%M:%S")
        datetime_list.append(formatted_datetime)
        
    dataframe["datetime"] = datetime_list
    dataframe["datetime"] = pd.to_datetime(dataframe["datetime"])
    dataframe["date"] = pd.to_datetime(dataframe["datetime"]).dt.date
    dataframe["time"] = pd.to_datetime(dataframe["datetime"]).dt.time
    dataframe["week_start"] = pd.to_datetime(dataframe["datetime"]).dt.to_period("W").dt.start_time.dt.date

    return dataframe                                      

In [13]:
# category
# item1 category dataframe
item1_cat = clean_catid(item1)
item1_cat = convert_to_datetime(item1_cat)

# item2 category dataframe
item2_cat = clean_catid(item2)
item2_cat = convert_to_datetime(item2_cat)

# combine item1 and item2 and drop columns
items_cat = pd.concat([item1_cat, item2_cat], axis=0)
items_cat = items_cat.drop(columns=["timestamp", "property", "time", "date", "datetime"])

# drop duplicates
items_cat = drop_duplicates(items_cat)

In [14]:
items_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 788214 entries, 0 to 361908
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   itemid      788214 non-null  int64 
 1   categoryid  788214 non-null  int64 
 2   week_start  788214 non-null  object
dtypes: int64(2), object(1)
memory usage: 24.1+ MB


In [15]:
items_cat.isnull().sum()

itemid        0
categoryid    0
week_start    0
dtype: int64

In [16]:
# display items_cat
items_cat.head()

,itemid,categoryid,week_start
0,460429,1338,2015-06-22
1,281245,1277,2015-05-18
2,35575,1059,2015-06-22
3,8313,1147,2015-07-13
4,55102,47,2015-07-20


In [17]:
print("Number of unique itemid:", len(items_cat["itemid"].unique()))
print("Number of unique categoryid:", len(items_cat["categoryid"].unique()))

Number of unique itemid: 417053
Number of unique categoryid: 1242


In [18]:
# some items have multiple categoryids
items_cat.groupby(["itemid"]).agg(cat_count = ("categoryid", "nunique")).\
sort_values(by="cat_count", ascending=False).head()

,cat_count
itemid,
42503,4
255468,4
391978,4
202195,4
231314,4


In [19]:
# categoryid changes for some itemids over time eg. 42503
items_cat_groups = items_cat.groupby("itemid")
items_cat.iloc[items_cat_groups.groups[42503], :].sort_values(by="week_start")

,itemid,categoryid,week_start
65267,42503,1167,2015-05-04
72831,42503,1167,2015-05-11
80396,42503,1167,2015-05-18
95657,42503,1167,2015-05-25
149470,42503,1167,2015-06-01
141716,42503,1167,2015-06-08
88051,42503,1167,2015-06-22
118663,42503,519,2015-06-29
103348,42503,519,2015-07-06
157204,42503,1574,2015-08-17


### Item - Availability

In [20]:
# availability
# item1 availability dataframe
item1_avail = clean_avail(item1)
item1_avail = convert_to_datetime(item1_avail)

# item2 category dataframe
item2_avail = clean_avail(item2)
item2_avail = convert_to_datetime(item2_avail)

# combine item1 and item2 and drop columns
items_avail = pd.concat([item1_avail, item2_avail], axis=0)
items_avail = items_avail.drop(columns=["timestamp", "property", "time", "date", "datetime"])

# drop duplicates
items_avail = drop_duplicates(items_avail)

In [21]:
# display items_cat
items_avail.head()

,itemid,is_available,week_start
0,285026,0,2015-06-29
1,186518,0,2015-07-13
2,423682,0,2015-06-01
3,316253,1,2015-06-08
4,430459,0,2015-07-13


In [22]:
# items availability 
items_avail["is_available"].value_counts(normalize=True)

0    0.573998
1    0.426002
Name: is_available, dtype: float64

In [23]:
items_avail.isnull().sum()

itemid          0
is_available    0
week_start      0
dtype: int64

In [24]:
# item availability for some itemids changes over time eg. 42503
items_avail_groups = items_avail.groupby("itemid")
items_avail.iloc[items_avail_groups.groups[42503], :].sort_values(by="week_start")

,itemid,is_available,week_start
355121,42503,1,2015-05-04
399680,42503,1,2015-05-11
384811,42503,1,2015-05-18
369956,42503,1,2015-05-25
414467,42503,1,2015-06-01
429330,42503,0,2015-06-08
532924,42503,1,2015-06-22
444173,42503,1,2015-06-29
458972,42503,1,2015-07-06
488519,42503,0,2015-08-17


### Item - Merge

In [25]:
# merge the 2 datasets
items_prop = items_cat.merge(items_avail, how="left", on=["itemid", "week_start"])

In [26]:
# some items with missing availability despite having data
items_prop_groups = items_prop.groupby("itemid")
items_prop.loc[items_prop_groups.groups[310832], 
               ["itemid", "categoryid", "week_start", "is_available"]].\
                sort_values(by = "week_start")

,itemid,categoryid,week_start,is_available
177538,310832,1404,2015-05-04,NaN
193072,310832,1404,2015-05-11,NaN
216355,310832,1404,2015-05-18,NaN
200836,310832,1404,2015-05-25,NaN
185273,310832,1404,2015-06-01,NaN
208557,310832,1404,2015-06-08,NaN
239534,310832,1404,2015-06-22,1.0
231783,310832,1690,2015-08-31,NaN
224057,310832,1690,2015-09-07,NaN


In [27]:
items_prop_chunk = []

for a_itemid in items_prop_groups.groups:
    a_item_df = items_prop.loc[items_prop_groups.groups[a_itemid], :].sort_values(by = "week_start")
    a_item_df[["is_available"]] = a_item_df[["is_available"]].\
    apply(lambda x: x.fillna(method="ffill").fillna(method="bfill")) # forward fill and backward fill
    
    if a_item_df["is_available"].isnull().sum() != 0:
        try:
            a_item_df[["is_available"]] = a_item_df[["is_available"]].\
            apply(lambda x: x.fillna(np.take(items_avail[items_avail["itemid"] == a_itemid].\
                                             sort_values(by="week_start")["is_available"][-1:].values, 0)))
           
            items_prop_chunk.append(a_item_df)
        except IndexError:
            items_prop_chunk.append(a_item_df)
            
    else:
        items_prop_chunk.append(a_item_df)

items_prop_clean = pd.concat(items_prop_chunk)

In [28]:
items_prop_clean = drop_duplicates(items_prop_clean)
items_prop_clean["is_available"] = items_prop_clean["is_available"].map(lambda x: int(float(x)))
items_prop_clean.head()

,itemid,categoryid,week_start,is_available
747073,0,209,2015-05-04,0
326394,1,1114,2015-05-04,1
354499,2,1305,2015-05-04,0
147267,3,1171,2015-05-25,0
768393,4,1038,2015-05-11,0


In [29]:
items_prop_clean.isnull().sum()

itemid          0
categoryid      0
week_start      0
is_available    0
dtype: int64

In [30]:
# save files to csv
items_cat.to_csv("../data/items_cat_clean.csv", index=False)
items_avail.to_csv("../data/items_avail_clean.csv", index=False)
items_prop_clean.to_csv("../data/items_prop_clean.csv", index=False)

## 3.2. Category Tree Dataset

This subsection covers the cleaning of the Category Tree dataset. Each row in the Category Tree dataset specifies an item category identifier and its corresponding parent identifier. There are some category id with no parent id. These rows with null values are dropped from the dataset.

In [31]:
# see null values
category_tree.isnull().sum()

categoryid     0
parentid      25
dtype: int64

In [32]:
# drop categories with no parent id
category_tree = category_tree.dropna(axis=0)

In [33]:
category_tree.isnull().sum()

categoryid    0
parentid      0
dtype: int64

In [34]:
print("Number of unique categoryid:", len(category_tree["categoryid"].unique()))
print("Number of unique parentid:", len(category_tree["parentid"].unique()))

Number of unique categoryid: 1644
Number of unique parentid: 362


In [35]:
# save file to csv
category_tree.to_csv("../data/category_tree_clean.csv", index=False)

## 3.3. Merged Items and Category Datasets

This sub-section covers the merging of the datasets from subsection 3.1 and 3.2, the merging of the item and its properties with its corresponding parent id. Items with no corresponding parent id will be dropped

In [36]:
items = items_prop_clean.merge(category_tree, how="left", on="categoryid")
items = items.dropna(axis=0)

In [37]:
items.head()

,itemid,categoryid,week_start,is_available,parentid
0,0,209,2015-05-04,0,293.0
1,1,1114,2015-05-04,1,113.0
2,2,1305,2015-05-04,0,1214.0
3,3,1171,2015-05-25,0,938.0
4,4,1038,2015-05-11,0,1174.0


In [38]:
# some items have multiple inputs and categoryids over time while some only have 1 
items.groupby(["itemid", "week_start"]).agg(cat_count = ("categoryid", "nunique")).\
sort_values(by="cat_count", ascending=False).head()

cat_count
itemid week_start           
0      2015-05-04          1
310731 2015-08-17          1
310804 2015-06-22          1
       2015-06-29          1
       2015-07-06          1

In [39]:
# change in categoryid, parentid snd availability for some itemids over time eg. 310804
items_groups = items.groupby("itemid")
items.loc[items_groups.groups[310804], :].sort_values(by="week_start")

,itemid,categoryid,week_start,is_available,parentid
525434,310804,1277,2015-05-04,0,312.0
525435,310804,1277,2015-05-11,1,312.0
525436,310804,1277,2015-05-18,1,312.0
525437,310804,1277,2015-05-25,1,312.0
525438,310804,1277,2015-06-01,1,312.0
525439,310804,1277,2015-06-08,1,312.0
525440,310804,1277,2015-06-22,1,312.0
525441,310804,1277,2015-06-29,1,312.0
525442,310804,1277,2015-07-06,1,312.0
525443,310804,1277,2015-07-13,0,312.0


In [40]:
items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 787996 entries, 0 to 788213
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   itemid        787996 non-null  int64  
 1   categoryid    787996 non-null  int64  
 2   week_start    787996 non-null  object 
 3   is_available  787996 non-null  int64  
 4   parentid      787996 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 52.2+ MB


In [41]:
# standardize the parentid (some have decimal points at the end)
# transform the datatype for parentid
def transform_int(dataframe, list_columns):
    for column in list_columns:
        dataframe[column] = dataframe[column].map(lambda x: int(float(x)))
    
    return dataframe

In [42]:
items = transform_int(items, ["parentid"])

In [43]:
print("Start weekdate of dataset:", items["week_start"].min())
print("End weekdate of dataset:", items["week_start"].max())

Start weekdate of dataset: 2015-05-04
End weekdate of dataset: 2015-09-07


In [44]:
# save file to csv
items.to_csv("../data/items_merge.csv", index=False)

## 3.4. Events Dataset

This sub-section covers the cleaning of the Events dataset. This dataset covers the behaviour data of the users to the website and contains hit-level interactions that include view, add to cart and transaction. 

Each row in the dataset indicates a user interaction on the site, whether the user is viewing, adding to the cart or purchasing a particular item. Rows that contain null values pertaining to transaction simply indicates that the users are not engaging in a transaction at that point of time. But instead, they could be viewing or adding to the cart a particular item.

In [45]:
# events datatype
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp      int64  
 1   visitorid      int64  
 2   event          object 
 3   itemid         int64  
 4   transactionid  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 105.1+ MB


In [46]:
# counts of each event type
events["event"].value_counts()

view           2664312
addtocart        69332
transaction      22457
Name: event, dtype: int64

In [47]:
# null count
events.isnull().sum()

timestamp              0
visitorid              0
event                  0
itemid                 0
transactionid    2733644
dtype: int64

In [48]:
def create_time_feats(dataframe, datetime_variable):
    dataframe["month"] = pd.to_datetime(dataframe[datetime_variable]).dt.month
    dataframe["day"] = pd.to_datetime(dataframe[datetime_variable]).dt.day
    dataframe["day_of_week"] = pd.to_datetime(dataframe[datetime_variable]).dt.dayofweek
    day_name = {1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday", 0: "Sunday"}
    dataframe["day_name"] = dataframe["day_of_week"].map(day_name)
    dataframe["day_name"] = pd.Categorical(dataframe["day_name"], 
                                           categories=["Monday", 
                                                       "Tuesday", 
                                                       "Wednesday", 
                                                       "Thursday", 
                                                       "Friday", 
                                                       "Saturday", 
                                                       "Sunday"], ordered=True)
    dataframe["hour"] = pd.to_datetime(dataframe[datetime_variable]).dt.hour

    return dataframe                                      

In [49]:
def transform_event_type(dataframe):
    dataframe["is_viewed"] = dataframe["event"].map(lambda x: 1 if x == "view" else 0)
    dataframe["is_added"] = dataframe["event"].map(lambda x: 1 if x == "addtocart" else 0)
    dataframe["is_purchased"] = dataframe["event"].map(lambda x: 1 if x == "transaction" else 0)
    
    return dataframe

In [50]:
# replace null with empty string
events["transactionid"] = events["transactionid"].fillna("")

# apply functions
events = drop_duplicates(events)
events_clean = convert_to_datetime(events)
events_clean = create_time_feats(events_clean, "datetime")
events_clean = transform_event_type(events_clean)

In [51]:
print("Start week date of dataset:", events_clean["week_start"].min())
print("End week date of dataset:", events_clean["week_start"].max())

Start week date of dataset: 2015-04-27
End week date of dataset: 2015-09-14


In [52]:
# save file to csv
events_clean.to_csv("../data/events_clean.csv", index=False)

## 3.5. Merged Events, Items and Category Datasets

This sub-section covers the merging of the combined items-category tree dataset from sub-section 3.3 and the events dataset from sub-section 3.4.

As the combined dataset is based on event-level (each row corresponds to one interaction for a particular user), the dataset would be further grouped so as to produce a session-based level dataset (each row corresponds to a combined list of a particular user's interactions during one session).

In the next notebook only the session-based level dataset will be used for further feature engineering, data visualisations and modelling.

### Event-Based

In [53]:
merge = events_clean.merge(items, how = "left", on = ["itemid", "week_start"])

In [54]:
merge.head()

,timestamp,visitorid,event,itemid,transactionid,datetime,date,time,week_start,month,day,day_of_week,day_name,hour,is_viewed,is_added,is_purchased,categoryid,is_available,parentid
0,1433221332117,257597,view,355908,,2015-06-02 08:02:12,2015-06-02,08:02:12,2015-06-01,6,2,1,Monday,8,1,0,0,NaN,NaN,NaN
1,1433224214164,992329,view,248676,,2015-06-02 08:50:14,2015-06-02,08:50:14,2015-06-01,6,2,1,Monday,8,1,0,0,NaN,NaN,NaN
2,1433221999827,111016,view,318965,,2015-06-02 08:13:19,2015-06-02,08:13:19,2015-06-01,6,2,1,Monday,8,1,0,0,NaN,NaN,NaN
3,1433221955914,483717,view,253185,,2015-06-02 08:12:35,2015-06-02,08:12:35,2015-06-01,6,2,1,Monday,8,1,0,0,NaN,NaN,NaN
4,1433221337106,951259,view,367447,,2015-06-02 08:02:17,2015-06-02,08:02:17,2015-06-01,6,2,1,Monday,8,1,0,0,1613.0,1.0,250.0


In [55]:
merge.isnull().sum()

timestamp              0
visitorid              0
event                  0
itemid                 0
transactionid          0
datetime               0
date                   0
time                   0
week_start             0
month                  0
day                    0
day_of_week            0
day_name               0
hour                   0
is_viewed              0
is_added               0
is_purchased           0
categoryid       2457683
is_available     2457683
parentid         2457683
dtype: int64

In [56]:
# some items with no categoryid, parentid and is_available despite having data
merge_groups = merge.groupby("itemid")
merge.loc[merge_groups.groups[310832], 
          ["visitorid", "itemid", "categoryid", "parentid", "is_available", "week_start"]].\
sort_values(by = "week_start")

,visitorid,itemid,categoryid,parentid,is_available,week_start
1476201,1281101,310832,1404.0,593.0,1.0,2015-05-04
1491325,71860,310832,1404.0,593.0,1.0,2015-05-04
1512346,397630,310832,1404.0,593.0,1.0,2015-05-04
1576097,828592,310832,1404.0,593.0,1.0,2015-05-04
1753154,655479,310832,1404.0,593.0,1.0,2015-05-18
1883978,1355459,310832,1404.0,593.0,1.0,2015-05-18
1893859,1355459,310832,1404.0,593.0,1.0,2015-05-18
1897619,1355459,310832,1404.0,593.0,1.0,2015-05-18
1930576,301102,310832,1404.0,593.0,1.0,2015-05-25
1934274,1225254,310832,1404.0,593.0,1.0,2015-05-25


In [57]:
items_chunk = []

for a_itemid in merge_groups.groups:
    a_item_df = merge.loc[merge_groups.groups[a_itemid], :].sort_values(by = "week_start")
    a_item_df[["categoryid", "parentid", "is_available"]] = a_item_df[["categoryid", "parentid", "is_available"]].\
    apply(lambda x: x.fillna(method="ffill").fillna(method="bfill"))
    
    if a_item_df["categoryid"].isnull().sum() != 0:
        try:
            a_item_df[["categoryid"]] = a_item_df[["categoryid"]].\
            apply(lambda x: x.fillna(np.take(items[items["itemid"] == a_itemid].\
                                             sort_values(by="week_start")["categoryid"][-1:].values, 0)))
            a_item_df[["parentid"]] = a_item_df[["parentid"]].\
            apply(lambda x: x.fillna(np.take(items[items["itemid"] == a_itemid].\
                                             sort_values(by="week_start")["parentid"][-1:].values, 0)))
            a_item_df[["is_available"]] = a_item_df[["is_available"]].\
            apply(lambda x: x.fillna(np.take(items[items["itemid"] == a_itemid].\
                                             sort_values(by="week_start")["is_available"][-1:].values, 0)))
    
    
            items_chunk.append(a_item_df)
        except IndexError:
            items_chunk.append(a_item_df)
            
    else:
        items_chunk.append(a_item_df)

events_merge = pd.concat(items_chunk)

In [58]:
# null values
events_merge.isnull().sum()

timestamp             0
visitorid             0
event                 0
itemid                0
transactionid         0
datetime              0
date                  0
time                  0
week_start            0
month                 0
day                   0
day_of_week           0
day_name              0
hour                  0
is_viewed             0
is_added              0
is_purchased          0
categoryid       255576
is_available     255576
parentid         255576
dtype: int64

In [59]:
# there are some items with no categoryid and purchaseid
# will drop them
events_merge = events_merge.dropna(axis=0)

In [60]:
# check data type
events_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500065 entries, 914759 to 1406152
Data columns (total 20 columns):
 #   Column         Dtype         
---  ------         -----         
 0   timestamp      int64         
 1   visitorid      int64         
 2   event          object        
 3   itemid         int64         
 4   transactionid  object        
 5   datetime       datetime64[ns]
 6   date           object        
 7   time           object        
 8   week_start     object        
 9   month          int64         
 10  day            int64         
 11  day_of_week    int64         
 12  day_name       category      
 13  hour           int64         
 14  is_viewed      int64         
 15  is_added       int64         
 16  is_purchased   int64         
 17  categoryid     float64       
 18  is_available   float64       
 19  parentid       float64       
dtypes: category(1), datetime64[ns](1), float64(3), int64(10), object(5)
memory usage: 383.9+ MB


In [61]:
# standardize the categoryid and parentid (some have decimal points at the end)
# transform the datatype for categoryid and parentid and is_available
events_merge = transform_int(events_merge, ["categoryid", "parentid", "is_available"])

# transform event type to categorical variable
events_merge["event"] = events_merge["event"].astype("category")

In [62]:
# check data type again
events_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500065 entries, 914759 to 1406152
Data columns (total 20 columns):
 #   Column         Dtype         
---  ------         -----         
 0   timestamp      int64         
 1   visitorid      int64         
 2   event          category      
 3   itemid         int64         
 4   transactionid  object        
 5   datetime       datetime64[ns]
 6   date           object        
 7   time           object        
 8   week_start     object        
 9   month          int64         
 10  day            int64         
 11  day_of_week    int64         
 12  day_name       category      
 13  hour           int64         
 14  is_viewed      int64         
 15  is_added       int64         
 16  is_purchased   int64         
 17  categoryid     int64         
 18  is_available   int64         
 19  parentid       int64         
dtypes: category(2), datetime64[ns](1), int64(13), object(4)
memory usage: 367.2+ MB


In [63]:
events_merge.head()

,timestamp,visitorid,event,itemid,transactionid,datetime,date,time,week_start,month,day,day_of_week,day_name,hour,is_viewed,is_added,is_purchased,categoryid,is_available,parentid
914759,1439922640493,370720,view,3,,2015-08-18 21:30:40,2015-08-18,21:30:40,2015-08-17,8,18,1,Monday,21,1,0,0,1171,0,938
1155833,1441031942792,639016,view,3,,2015-08-31 17:39:02,2015-08-31,17:39:02,2015-08-31,8,31,0,Sunday,17,1,0,0,1171,0,938
601537,1435647791545,1042455,view,4,,2015-06-30 10:03:11,2015-06-30,10:03:11,2015-06-29,6,30,1,Monday,10,1,0,0,1038,0,1174
1159185,1441044360244,905555,view,4,,2015-08-31 21:06:00,2015-08-31,21:06:00,2015-08-31,8,31,0,Sunday,21,1,0,0,1038,0,1174
1433025,1442359364099,1010132,view,4,,2015-09-16 02:22:44,2015-09-16,02:22:44,2015-09-14,9,16,2,Tuesday,2,1,0,0,1038,0,1174


In [64]:
# save file to csv
events_merge.to_csv("../data/events_merge.csv", index=False)

### Session-Based

In [65]:
time_out = timedelta(minutes=30)
visitorid_group = events_merge.groupby("visitorid")

# transform the interaction-based data to session-based
sessions_list = []
for a_visitorid in visitorid_group.groups:
    a_visitor_df = events_merge.loc[visitorid_group.groups[a_visitorid], :].sort_values("datetime")
    
    if not a_visitor_df.empty:
        start_session = a_visitor_df.iloc[0, :]["datetime"]
        visitorid = a_visitorid
        items_dict = dict([(i, []) for i in events_merge["event"].cat.categories])
        cat_dict = dict([(i, []) for i in events_merge["event"].cat.categories])
        parent_dict = dict([(i, []) for i in events_merge["event"].cat.categories])
        avail_dict = dict([(i, 0) for i in events_merge["event"].cat.categories])
        
        for index, row in a_visitor_df.iterrows():
            if row["datetime"] - start_session <= time_out:
                items_dict[row["event"]].append(row["itemid"])
                cat_dict[row["event"]].append(row["categoryid"])
                parent_dict[row["event"]].append(row["parentid"])
                avail_dict[row["event"]] += row["is_available"]
                end_session = row["datetime"]
                session_duration = pd.to_datetime(end_session) - pd.to_datetime(start_session)
                session_duration_s = session_duration.total_seconds()
                session_duration_m = round(session_duration_s / 60, 2)
            
            else:
                sessions_list.append([visitorid, start_session, end_session, 
                                      session_duration_s, session_duration_m] + 
                                    [value for key, value in items_dict.items()] + 
                                    [value for key, value in cat_dict.items()] + 
                                    [value for key, value in parent_dict.items()] +
                                    [value for key, value in avail_dict.items()])
                
                start_session = row["datetime"]
                items_dict = dict([(i, []) for i in events_merge["event"].cat.categories])
                cat_dict = dict([(i, []) for i in events_merge["event"].cat.categories])
                parent_dict = dict([(i, []) for i in events_merge["event"].cat.categories])
                avail_dict = dict([(i, 0) for i in events_merge["event"].cat.categories])
                end_session = row["datetime"]
                session_duration = pd.to_datetime(end_session) - pd.to_datetime(start_session)
                session_duration_s = session_duration.total_seconds()
                session_duration_m = round(session_duration_s / 60, 2)
                
        incomplete_session = False
        for key, value in items_dict.items():
            if value:
                incomplete_session=True
                break
        
        if incomplete_session:
            sessions_list.append([visitorid, start_session, end_session, 
                                  session_duration_s, session_duration_m] + 
                                [value for key, value in items_dict.items()] + 
                                [value for key, value in cat_dict.items()] + 
                                [value for key, value in parent_dict.items()] + 
                                [value for key, value in avail_dict.items()])

In [66]:
# transform session-based list into a dataframe
sessions = pd.DataFrame(sessions_list, 
                       columns=["visitorid", 
                                "start_session", 
                                "end_session",
                                "session_duration_s", 
                                "session_duration_m",
                                "cart", 
                                "transaction", 
                                "view", 
                                "cart_cat", 
                                "transaction_cat", 
                                "view_cat", 
                                "cart_parent", 
                                "transaction_parent", 
                                "view_parent", 
                                "cart_avail", 
                                "transaction_avail", 
                                "view_avail"])

In [67]:
# display row
sessions.head()

,visitorid,start_session,end_session,session_duration_s,session_duration_m,cart,transaction,view,cart_cat,transaction_cat,view_cat,cart_parent,transaction_parent,view_parent,cart_avail,transaction_avail,view_avail
0,0,2015-09-11 23:49:49,2015-09-11 23:55:17,328.0,5.47,[],[],"[285930, 357564, 67045]",[],[],"[1188, 256, 333]",[],[],"[1497, 1257, 1497]",0,0,2
1,1,2015-08-13 20:46:06,2015-08-13 20:46:06,0.0,0.00,[],[],[72028],[],[],[1192],[],[],[955],0,0,0
2,2,2015-08-07 20:51:44,2015-08-07 21:20:57,1753.0,29.22,[],[],"[325215, 325215, 259884, 216305, 342816, 34281...",[],[],"[299, 299, 299, 299, 444, 444, 299, 299]",[],[],"[73, 73, 73, 73, 73, 73, 73, 73]",0,0,4
3,3,2015-08-01 10:10:35,2015-08-01 10:10:35,0.0,0.00,[],[],[385090],[],[],[1171],[],[],[938],0,0,0
4,5,2015-07-17 04:45:56,2015-07-17 04:45:56,0.0,0.00,[],[],[61396],[],[],[646],[],[],[1606],0,0,0


In [68]:
# add new features to the dataset
def add_features(dataframe):
    columns = ["view", "view_cat", "view_parent", 
               "cart", "cart_cat", "cart_parent", 
               "transaction", "transaction_cat",  "transaction_parent"]  
    
    column_name = ""
    for i, column in enumerate(columns):
        column_name1 = "tot_" + column
        column_name2 = "tot_unique_" + column
        dataframe[column_name1] = dataframe[column].map(lambda x: len(x))
        dataframe[column_name2] = dataframe[column].map(lambda x: len(set(x)))
   
    return dataframe

In [69]:
sessions_add = create_time_feats(sessions, "start_session")
sessions_add = add_features(sessions)
sessions_add["week_start"] = pd.to_datetime(sessions_add["start_session"]).dt.to_period("W").dt.start_time.dt.date

# remove rows where users have no interactions
sessions_add = sessions_add[~((sessions_add["tot_view"] == 0) & 
                              (sessions_add["tot_cart"] == 0) & 
                              (sessions_add["tot_transaction"] == 0))]

In [70]:
sessions_add.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1349015 entries, 0 to 1454754
Data columns (total 41 columns):
 #   Column                         Non-Null Count    Dtype         
---  ------                         --------------    -----         
 0   visitorid                      1349015 non-null  int64         
 1   start_session                  1349015 non-null  datetime64[ns]
 2   end_session                    1349015 non-null  datetime64[ns]
 3   session_duration_s             1349015 non-null  float64       
 4   session_duration_m             1349015 non-null  float64       
 5   cart                           1349015 non-null  object        
 6   transaction                    1349015 non-null  object        
 7   view                           1349015 non-null  object        
 8   cart_cat                       1349015 non-null  object        
 9   transaction_cat                1349015 non-null  object        
 10  view_cat                       1349015 non-null  objec

In [71]:
# save file to csv
sessions_add.to_csv("../data/sessions.csv", index=False)

### User-Based

In [72]:
visitorid_group = sessions_add.groupby("visitorid")

# transform session-based data to user-based
users_list = []
for a_visitorid in visitorid_group.groups:
    a_visitorid_df = sessions.loc[visitorid_group.groups[a_visitorid], :]
    
    if not a_visitorid_df.empty:
        visitorid = a_visitorid
        tot_sessions = 0
        tot_duration_s = 0
        addtocart = []
        transaction = []
        view = []
        addtocart_cat = []
        transaction_cat = []
        view_cat = []
        addtocart_parent = []
        transaction_parent = []
        view_parent = []
       
        for index, row in a_visitorid_df.iterrows():
            addtocart.extend(row["cart"])
            transaction.extend(row["transaction"])
            view.extend(row["view"])
            addtocart_cat.extend(row["cart_cat"])
            transaction_cat.extend(row["transaction_cat"])
            view_cat.extend(row["view_cat"])
            addtocart_parent.extend(row["cart_parent"])
            transaction_parent.extend(row["transaction_parent"])
            view_parent.extend(row["view_parent"])
            tot_sessions += 1
            tot_duration_s += row["session_duration_s"]
        
        avg_session_duration = round((tot_duration_s / tot_sessions) / 60, 2)
            
        users_list.append([visitorid, tot_sessions, tot_duration_s,
                           avg_session_duration, view, addtocart, 
                           transaction, addtocart_cat, transaction_cat, 
                           view_cat, addtocart_parent, transaction_parent, 
                           view_parent])

In [73]:
# transform user-based list into a dataframe
users = pd.DataFrame(users_list, 
                     columns = ["visitorid", "tot_sessions", "tot_duration_s",
                                "avg_session_duration", "view", "cart", 
                                "transaction", "cart_cat", "transaction_cat", 
                                "view_cat", "cart_parent", "transaction_parent", 
                                "view_parent"])

In [74]:
users.head()

,visitorid,tot_sessions,tot_duration_s,avg_session_duration,view,cart,transaction,cart_cat,transaction_cat,view_cat,cart_parent,transaction_parent,view_parent
0,0,1,328.0,5.47,"[285930, 357564, 67045]",[],[],[],[],"[1188, 256, 333]",[],[],"[1497, 1257, 1497]"
1,1,1,0.0,0.00,[72028],[],[],[],[],[1192],[],[],[955]
2,2,1,1753.0,29.22,"[325215, 325215, 259884, 216305, 342816, 34281...",[],[],[],[],"[299, 299, 299, 299, 444, 444, 299, 299]",[],[],"[73, 73, 73, 73, 73, 73, 73, 73]"
3,3,1,0.0,0.00,[385090],[],[],[],[],[1171],[],[],[938]
4,5,1,0.0,0.00,[61396],[],[],[],[],[646],[],[],[1606]


In [75]:
# apply function to add features to the dataset
users_add = add_features(users)

# create total score
# view an item - 1 point, add an item to the cart - 5 points, purchase an item - 50 points
users_add["tot_score"] = users_add["tot_view"] +\
                        (users_add["tot_cart"] * 5) +\
                        (users_add["tot_transaction"] * 50)

In [76]:
users_add.head()

,visitorid,tot_sessions,tot_duration_s,avg_session_duration,view,cart,transaction,cart_cat,transaction_cat,view_cat,cart_parent,transaction_parent,view_parent,tot_view,tot_unique_view,tot_view_cat,tot_unique_view_cat,tot_view_parent,tot_unique_view_parent,tot_cart,tot_unique_cart,tot_cart_cat,tot_unique_cart_cat,tot_cart_parent,tot_unique_cart_parent,tot_transaction,tot_unique_transaction,tot_transaction_cat,tot_unique_transaction_cat,tot_transaction_parent,tot_unique_transaction_parent,tot_score
0,0,1,328.0,5.47,"[285930, 357564, 67045]",[],[],[],[],"[1188, 256, 333]",[],[],"[1497, 1257, 1497]",3,3,3,3,3,2,0,0,0,0,0,0,0,0,0,0,0,0,3
1,1,1,0.0,0.00,[72028],[],[],[],[],[1192],[],[],[955],1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2,1,1753.0,29.22,"[325215, 325215, 259884, 216305, 342816, 34281...",[],[],[],[],"[299, 299, 299, 299, 444, 444, 299, 299]",[],[],"[73, 73, 73, 73, 73, 73, 73, 73]",8,4,8,2,8,1,0,0,0,0,0,0,0,0,0,0,0,0,8
3,3,1,0.0,0.00,[385090],[],[],[],[],[1171],[],[],[938],1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4,5,1,0.0,0.00,[61396],[],[],[],[],[646],[],[],[1606],1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1


In [77]:
# save file to csv
users_add.to_csv("../data/users.csv", index=False)